# Notes

* Need to work out pagination in queries, and how to recognize current data length such that only query new records 

In [ ]:
# %pip install flipside

# Imports

In [ ]:
from flipside import Flipside
from config import flipside_api_key, flipside_nft_holder_address

In [ ]:
import pandas as pd
import os

In [ ]:
sdk = Flipside(flipside_api_key)

# Variables

In [ ]:
# title = "_may_12"

# Define Save Function

In [ ]:
data_path = "/app/data/source"

def get_cwd():
    cwd_temp = os.getcwd()
    temp_split = cwd_temp.split('/')
    cwd = ""
    for x in temp_split:
        if x == 'experiments':
            break
        elif x == '':
            pass
        else:
            cwd += "/"+x       
    return cwd


def print_metrics(query):
    started_at = query.run_stats.started_at
    ended_at = query.run_stats.ended_at
    elapsed_seconds = query.run_stats.elapsed_seconds
    record_count = query.run_stats.record_count
    print(f"This query took ${elapsed_seconds} seconds to run and returned {record_count} records from the database.")

def query_and_save(_query, _filename, _page_size = 30000):
    # Initial Query
    print(f"___\n{_filename}")
    print(f"querying page: 1")
    query_result_set = sdk.query(_query, page_size = _page_size)
    df_output = pd.json_normalize(query_result_set.records)
  
    # Metrics
    print_metrics(query_result_set)
    
    # Handle Pagination
    if len(query_result_set.records) >= _page_size:
        i = 2
        keep_going = True
        while keep_going:
            print(f"querying page: {i}")
            extended_result_set = sdk.get_query_results(
                query_result_set.query_id,
                page_number=i,
                page_size=_page_size
            )
            # Metrics
            print_metrics(query_result_set)
            
            # Concat Dataframes
            df_local = pd.json_normalize(extended_result_set.records)
            df_output = pd.concat([df_output, df_local]).reset_index()
            
            # Check if continue
            print(len(extended_result_set.records) < _page_size)
            print(len(extended_result_set.records))
            if len(extended_result_set.records) < _page_size:
                keep_going = False
            i += 1
    # Save
    cwd = get_cwd()
    full_filename = cwd+ data_path + '/' + filename+'.csv'
    df_output.to_csv(full_filename) 
    
    return df_output

# Defining SQL

## Curve Locker

In [ ]:
curve_locker_address = '0x5f3b5DfEb7B28CDbD7FAba78963EE202a494e2A2'

curve_locker_query = f"""SELECT 
  *,
  WEEK(BLOCK_TIMESTAMP) as WEEK_NUMBER,
  DAYOFWEEK(BLOCK_TIMESTAMP) as WEEK_DAY

FROM ethereum.core.ez_decoded_event_logs
WHERE CONTRACT_ADDRESS = lower('{curve_locker_address}')
"""
# AND BLOCK_TIMESTAMP <'2022-01-01 00:00:00.000'


In [ ]:
filename = 'curve_locker'
df_curve_locker = query_and_save(curve_locker_query, filename)

## Curve Gauge Votes

In [ ]:
curve_voter_address = '0x2F50D538606Fa9EDD2B11E2446BEb18C9D5846bB'

curve_gauge_vote_query = f"""SELECT 
    SYMBOL, 
    NAME, 
    WEEK(BLOCK_TIMESTAMP) as WEEK_NUMBER,
    DAYOFWEEK(BLOCK_TIMESTAMP) as WEEK_DAY,
    DECODED_LOG,
    TX_HASH,
    BLOCK_TIMESTAMP

FROM ethereum.core.ez_decoded_event_logs LOGS
 LEFT JOIN ethereum.core.dim_contracts CONTRACT
   ON CONTRACT.address = lower(LOGS.DECODED_LOG:gauge_addr::string)
WHERE CONTRACT_ADDRESS = lower('{curve_voter_address}')
AND EVENT_NAME = 'VoteForGauge'
ORDER BY BLOCK_TIMESTAMP ASC
"""

In [ ]:
filename = 'curve_gauge_votes'
df_curve_locker = query_and_save(curve_gauge_vote_query, filename, 10000)

## Snapshot Votes

In [ ]:
convex_snapshot_votes_query = f"""
SELECT 
    PROPOSAL_ID, 
    PROPOSAL_START_TIME, 
    PROPOSAL_END_TIME, 
    PROPOSAL_TITLE, 
    PROPOSAL_AUTHOR,
    VOTE_OPTION,
    VOTING_POWER,
    VOTE_TIMESTAMP,
    QUORUM,
    CHOICES,
    VOTING_PERIOD,
    NETWORK,
    SPACE_ID,
    VOTER,
    ADDRESS_NAME,
    LABEL_TYPE, 
    LABEL_SUBTYPE,
    LABEL
FROM ethereum.core.ez_snapshot SNAPSHOT
 LEFT JOIN ethereum.core.dim_labels LABELS
   ON SNAPSHOT.VOTER = LABELS.ADDRESS
WHERE SPACE_ID = 'cvx.eth' 
AND PROPOSAL_TITLE LIKE 'Gauge Weight%'
AND VOTE_TIMESTAMP > '2022-12-20 00:00:00.000'
"""

In [ ]:
filename = 'convex_snapshot_votes'
df_snapshot_votes = query_and_save(convex_snapshot_votes_query, filename)

## Reference: Gauge to LP map

In [ ]:
curve_voter_address = '0x2F50D538606Fa9EDD2B11E2446BEb18C9D5846bB'

gauge_to_lp_map_query = """
with v2_deployer as (
  SELECT
    BLOCK_TIMESTAMP as block_timestamp,
    DECODED_LOG:gauge::string as gauge_addr,
    DECODED_LOG:pool::string as pool_addr,
    DECODED_LOG:token::string as token_addr,
    'v2' as type
  
  
  FROM ethereum.core.ez_decoded_event_logs
  WHERE CONTRACT_ADDRESS = lower('0xF18056Bbd320E96A48e3Fbf8bC061322531aac99') -- v2 deployer
  AND EVENT_NAME = 'LiquidityGaugeDeployed'

),

factory_deployer as (
  SELECT 
    BLOCK_TIMESTAMP as block_timestamp,
    DECODED_LOG:gauge::string as gauge_addr,
    DECODED_LOG:pool::string as pool_addr,
    '' as token_addr,
    'factory' as type

  FROM ethereum.core.ez_decoded_event_logs as logs

  WHERE logs.CONTRACT_ADDRESS = lower('0xB9fC157394Af804a3578134A6585C0dc9cc990d4')  -- factory
  AND logs.EVENT_NAME = 'LiquidityGaugeDeployed'
),


combo as (
  SELECT gauge_addr, pool_addr, token_addr, type, block_timestamp FROM v2_deployer
  UNION
  SELECT gauge_addr, pool_addr, token_addr, type, block_timestamp  FROM factory_deployer
)

SELECT 
  combo.gauge_addr as GAUGE_ADDR, 
  combo.pool_addr as POOL_ADDR, 
  combo.token_addr as TOKEN_ADDR, 
  combo.type as TYPE, 
  combo.block_timestamp as BLOCK_TIMESTAMP,
  contracts.NAME as GAUGE_NAME,
  contracts.SYMBOL as GAUGE_SYMBOL,
  pool_contracts.NAME as POOL_NAME,
  pool_contracts.SYMBOL as POOL_SYMBOL,
  token_contracts.NAME as TOKEN_NAME,
  token_contracts.SYMBOL as TOKEN_SYMBOL
FROM combo
LEFT JOIN ethereum.core.dim_contracts as contracts
  ON combo.gauge_addr = contracts.ADDRESS
LEFT JOIN ethereum.core.dim_contracts as pool_contracts
  ON combo.pool_addr = pool_contracts.ADDRESS
LEFT JOIN ethereum.core.dim_contracts as token_contracts
  ON combo.token_addr = token_contracts.ADDRESS
"""

In [ ]:
filename = 'gauge_to_lp_map'
df_curve_locker = query_and_save(gauge_to_lp_map_query, filename)